# Data cleaning and preparation
Before we jump into the EDA (explortory data analysis), we need to get our data cleaned and ready.

# Where is the data from?
This project uses data that has been scraped from Board Game Geek (https://boardgamegeek.com) and made available on Kaggle.

**Source** [Board Game Reviews - Jan 2025] https://www.kaggle.com/datasets/bwandowando/boardgamegeek-board-games-reviews-jan-2025

**Author Credit** [bwandowando] https://www.kaggle.com/bwandowando

# About the raw data
The dataset consists of 4 csv files with some of the fields shown below:
- boardgames (Titles, Descriptions, Release Year, Ratings)
- boardgames_reviews (Pseudo Users, Comments, Post Date, Rating)
- users (Pseudo Users, Location, Badges)
- user_game_status (Pseudo Users, Ownership, Wishlisted)

Before loading the raw data into Jupyter Notebooks, I completed some intial data cleaining in SQL.

---


## boardgames.csv
The raw boardgames csv file initially looks like this:

![Raw_boardgames_file](../images/Raw_boardgames_file.png)

# The changes I made to this file are:
- Renamed column headers (more for personal preference, but it also helps me to remember w
- hat fields I have)
- Removed "link" and "thumbnail" as I don't currently need them

I then ran the below SQL code:
![boardgames_file_SQL_code](../images/boardgames_file_SQL_code.png)

Which can be copied below:

SELECT
	a."Rank",
	a."Game ID",
	a."Title",
	a."Description",
	a."Release Year",
	a."Geek Rating",
	a."Avg Rating",
	a."Voters"
FROM
	( 	SELECT 
			ROW_NUMBER () OVER ( PARTITION BY a."Game ID" ORDER BY a."Game ID" ) AS "Rn",
			a."Rank",
			a."Game ID",
			a."Title",
			a."Description",
			a."Year" AS "Release Year",
			a."Geek Rating",
			a."Avg Rating",
			a."Voters"	
		FROM 	
			dbo."BPP - Board Games" a
		WHERE 	
			a."Geek Rating" IS NOT NULL 
		AND 	
			a."Year" IS NOT NULL 		
		AND		
			a."Game ID" IS NOT NULL 				
		AND 	
			a."Rank" IS NOT NULL		
		AND
		 	a."Year" < 2024
	) a
WHERE a."Rn" = 1 
ORDER BY a."Rank"

# What does this do?

- The Raw file consists of 161,404 "board games". I use quotes here as I would argue not every entry is actually a board game.
- Firstly, any game without a "Geek Rating" is removed. More details about the Geek rating are below. This reduce our list of games from 161,404 to 38,059.
- Some games do not have a release year. Perhaps no one knows when these games were released. Anyway, they were removed from the dataset reducing the total by a further 276 to 37,783 games. Games without a release year included things like Go Fish and Poker Dice etc.
- Some games did not feature a "Game ID" or a "Rank". These were often game expansions and second editions. These were all excluded removing a further 10,943 leaving 26,840. That's still a lot of games!
- Any duplicate "Game ID" were also removed, just in case, but there were no duplicates, but seemed silly to remove the code which was doing no harm.
- Lastly, I removed any games released in 2024. This is because I want to work with full years worth of data. This ensured every game on the list can contain at least one years worth of data. This removed another 1,045 games.

This leaves our starting pot of games at...
# 25,795

# What is a Geek Rating?
To receive a geek rating, a game must have at least 30 votes (or reviews)
